In [1141]:
import torchvision.models as models
model = models.vgg19(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
print(model)

/Users/kiranmanicka/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/kiranmanicka/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [1142]:
import torch 
import PIL
from PIL import Image 
import torchvision.transforms as transforms
import torch.nn as nn

In [1143]:

class Normalization(nn.Module):
    def __init__(self):
        super(Normalization, self).__init__()
        # .view the mean and std to make them [C x 1 x 1] so that they can
        # directly work with image Tensor of shape [B x C x H x W].
        # B is batch size. C is number of channels. H is height and W is width.
        self.mean = torch.tensor([0.485, 0.456, 0.406]).view(-1,1,1)
        self.std = torch.tensor([0.229, 0.224, 0.225]).view(-1,1,1)

    def forward(self, img):
        # normalize ``img``
        return (img - self.mean) / self.std




class mod_vgg(nn.Module):
    def __init__(self,model,index):
        super().__init__()
        l=[]
        i=0
        for r in model.children():
            if i==index:
                l.append(nn.Flatten(start_dim=0, end_dim=-1))
            l.append(r)
            i+=1
        self.layers = nn.ModuleList(l)
        

    def forward(self, x):
        # ModuleList can act as an iterable, or be indexed using ints
        for r  in self.layers:
            x=r(x)
            print(x.size())
        return x

In [1144]:
content_directory='content'
style_directory='style'

content_image = Image.open(f'{content_directory}/lion.jpeg') 
style_image=Image.open(f'{style_directory}/wave.jpeg')


# Define a transform to convert PIL  
# image to a Torch tensor 
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    
]) 
  
# transform = transforms.PILToTensor() 
# Convert the PIL image to Torch tensor 
content_tensor = transform(content_image)
style_tensor=transform(style_image)


tensor([[[0.9882, 0.9922, 0.9922,  ..., 0.9961, 0.9961, 0.9961],
         [0.9686, 0.9725, 0.9686,  ..., 0.9843, 0.9882, 0.9882],
         [0.9647, 0.9333, 0.8784,  ..., 0.9686, 0.9765, 0.9686],
         ...,
         [0.9059, 0.3843, 0.2431,  ..., 0.9922, 0.9922, 0.9961],
         [0.9608, 0.8941, 0.8471,  ..., 0.9922, 0.9922, 0.9961],
         [0.9882, 0.9882, 0.9843,  ..., 0.9922, 0.9922, 0.9961]],

        [[0.9882, 0.9882, 0.9922,  ..., 0.9922, 0.9961, 0.9961],
         [0.9725, 0.9804, 0.9765,  ..., 0.9843, 0.9882, 0.9882],
         [0.9765, 0.9490, 0.8980,  ..., 0.9725, 0.9765, 0.9686],
         ...,
         [0.8980, 0.3765, 0.2392,  ..., 0.9922, 0.9922, 0.9961],
         [0.9569, 0.8902, 0.8471,  ..., 0.9922, 0.9922, 0.9961],
         [0.9882, 0.9882, 0.9843,  ..., 0.9922, 0.9922, 0.9961]],

        [[0.9882, 0.9843, 0.9765,  ..., 0.9922, 0.9961, 0.9961],
         [0.9725, 0.9765, 0.9765,  ..., 0.9843, 0.9882, 0.9882],
         [0.9725, 0.9569, 0.9176,  ..., 0.9725, 0.9765, 0.

In [1145]:
print(content_tensor.size())

torch.Size([3, 128, 128])


In [1146]:
model.children()

<generator object Module.children at 0x3ec2efc10>

In [1147]:
mod_model=mod_vgg(model,2)

Figure out how to grab high level activations and gram matrices 

In [1148]:
print(torch.sum(content_tensor))

tensor(24032.2539)


In [1149]:
content_loss_index=8
style_loss_indices=[1,3,6,8,11]


temp_layers=list(list(list(mod_model.children())[0].children())[0])
temp_layers.insert(0,Normalization())


def get_activations(image):
    base_content_activation=None
    base_style_activation=[]
    for r in range(len(temp_layers)):
        image=temp_layers[r](image)
        if r==content_loss_index:
            base_content_activation=image.clone()
        if r in style_loss_indices:
            base_style_activation.append(image.clone())
            if style_loss_indices[-1]==r:
                break
    return base_content_activation,base_style_activation

model.requires_grad_(False)
for r in temp_layers:
    r.requires_grad_(False)


In [1150]:
base_content,_=get_activations(content_tensor)
_,base_style=get_activations(style_tensor)

def calc_gram_matrix(feature_maps):
    a,b,c=feature_maps.size()
    features = feature_maps.view(a, b*c)
    G = torch.mm(features, features.t())
    #return G
    return G.div(a * b * c)
    
#     return tensor

base_gram_matrices=[calc_gram_matrix(r) for r in base_style]

[tensor([[ 0.0516,  0.0004,  0.0712,  ..., -0.0217, -0.0094,  0.0009],
         [ 0.0004,  0.0991, -0.0225,  ..., -0.0075, -0.0130,  0.0164],
         [ 0.0712, -0.0225,  0.1244,  ..., -0.0500, -0.0142, -0.0076],
         ...,
         [-0.0217, -0.0075, -0.0500,  ...,  0.0330,  0.0085,  0.0048],
         [-0.0094, -0.0130, -0.0142,  ...,  0.0085,  0.0115, -0.0037],
         [ 0.0009,  0.0164, -0.0076,  ...,  0.0048, -0.0037,  0.0254]]),
 tensor([[ 0.2368,  0.0605,  0.0519,  ...,  0.0856,  0.1101, -0.0101],
         [ 0.0605,  0.0705,  0.0293,  ...,  0.0008,  0.0915,  0.0150],
         [ 0.0519,  0.0293,  0.0786,  ...,  0.0250,  0.0962, -0.0064],
         ...,
         [ 0.0856,  0.0008,  0.0250,  ...,  0.1292,  0.0239, -0.0250],
         [ 0.1101,  0.0915,  0.0962,  ...,  0.0239,  0.2624,  0.0453],
         [-0.0101,  0.0150, -0.0064,  ..., -0.0250,  0.0453,  0.2592]]),
 tensor([[ 0.7251,  0.0134,  0.1119,  ..., -0.2157,  0.2948,  0.2464],
         [ 0.0134,  0.2292,  0.0048,  ...,  0

Construct loss function

In [1151]:
#detaching the base tensors
base_content=base_content.detach()
for r in range(len(base_style)):
    base_style[r]=base_style[r].detach()

In [1152]:
#loss function
def overall_loss(img,content, style,base_content,base_style,version='both',variation=True):
    def total_variation_loss(img, weight):
        c_img, h_img, w_img = img.size()
        tv_h = torch.pow(img[:,1:,:]-img[:,:-1,:], 2).sum()
        tv_w = torch.pow(img[:,:,1:]-img[:,:,:-1], 2).sum()
        return weight*(tv_h+tv_w)/(c_img*h_img*w_img)
    if variation:
        variation_loss=total_variation_loss(img,1000000)
    else:
        torch.tensor(0)
    style_loss=nn.MSELoss()
    content_loss=nn.MSELoss()
    alpha=1
    beta=100000
    style_loss_tensor=torch.tensor(0)
    content_loss_tensor=torch.tensor(0)
    for r in range(len(base_style)):
        style_loss_tensor=style_loss_tensor+ style_loss(calc_gram_matrix(style[r]),base_gram_matrices[r])
    
    content_loss_tensor=content_loss(content,base_content)
    if version=='style':
        return style_loss_tensor+variation_loss
    if version=='content':
        return content_loss_tensor+variation_loss
    
    return alpha*content_loss_tensor + beta * style_loss_tensor

Gradient Descent Step

In [1153]:
iterations=300

In [1154]:
import torch.optim as optim

generated_image=content_tensor.clone()
# generated_image = generated_image.to(torch.float)
generated_image.requires_grad=True
loss_list=[100,90]

LBFGS_optimizer=optim.LBFGS([generated_image])
optimizer=optim.SGD([generated_image],lr=.1)

test_clone=generated_image.clone().detach()
threshold=0
i=[0]
while i[0]<iterations:
    # making predictions with forward pass
    print(i)
    def closure():
        with torch.no_grad():
            generated_image.clamp_(0, 1)
        i[0]+=1
        content,style=get_activations(generated_image)
        # calculating the loss between original and predicted data points
        computed_loss=overall_loss(generated_image,content,style,base_content,base_style,"both",variation="False")
        LBFGS_optimizer.zero_grad()
        # storing the calculated loss in a list
        loss_list.append(computed_loss.item())
       
        computed_loss.backward()
        return computed_loss
    # updateing the parameters after each iteration
    LBFGS_optimizer.step(closure)
    
    

[0]
[20]
[40]
[60]
[80]
[100]
[120]
[140]
[160]
[180]
[200]
[220]
[240]
[260]
[280]


In [1155]:
invTrans2 = transforms.Compose([ 
                                transforms.ToPILImage()
                                
                               ])
print(generated_image)
with torch.no_grad():
    generated_image.clamp_(0, 1)
inversed_image=invTrans2(generated_image)
inversed_image.show()

tensor([[[0.6145, 0.6231, 0.7462,  ..., 0.3572, 0.1931, 0.1404],
         [0.6730, 0.4451, 0.3993,  ..., 0.5510, 0.4929, 0.5342],
         [0.5089, 0.5272, 0.4173,  ..., 0.5556, 0.5712, 0.5636],
         ...,
         [0.8573, 1.0026, 0.4742,  ..., 0.7108, 0.6983, 0.5989],
         [0.7829, 0.9751, 0.9938,  ..., 0.7810, 0.6692, 0.6075],
         [0.9225, 0.7345, 0.8518,  ..., 0.8949, 0.7889, 0.6343]],

        [[0.4075, 0.5868, 0.8685,  ..., 0.3734, 0.3525, 0.2081],
         [0.5897, 0.8139, 0.7864,  ..., 0.7420, 0.7712, 0.7654],
         [0.6690, 0.6813, 0.7531,  ..., 0.6580, 0.6364, 0.5689],
         ...,
         [0.8175, 1.0007, 0.6955,  ..., 0.8735, 0.6842, 0.7278],
         [0.5757, 0.9939, 0.9869,  ..., 0.9134, 0.6842, 0.6702],
         [0.5045, 0.7174, 0.9671,  ..., 0.9908, 0.6384, 0.5971]],

        [[0.5351, 0.7084, 0.6970,  ..., 0.4783, 0.3357, 0.3484],
         [0.8030, 0.7023, 0.6863,  ..., 0.7174, 0.6495, 0.6373],
         [0.8450, 0.7559, 0.8462,  ..., 0.8307, 0.7674, 0.